In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 8: Matchsticks ---</h2><p>Space on the sleigh is limited this year, and so Santa will be bringing his list as a digital copy. He needs to know how much space it will take up when stored.</p>
<p>It is common in many programming languages to provide a way to <span title="It is common for many programmers to try to escape from string escaping.  No such luck here.">escape</span> special characters in strings.  For example, <a href="https://en.wikipedia.org/wiki/Escape_sequences_in_C">C</a>, <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/String">JavaScript</a>, <a href="http://perldoc.perl.org/perlop.html#Quote-and-Quote-like-Operators">Perl</a>, <a href="https://docs.python.org/2.0/ref/strings.html">Python</a>, and even <a href="http://php.net/manual/en/language.types.string.php#language.types.string.syntax.double">PHP</a> handle special characters in very similar ways.</p>
<p>However, it is important to realize the difference between the number of characters <em>in the code representation of the string literal</em> and the number of characters <em>in the in-memory string itself</em>.</p>
<p>For example:</p>
<ul>
<li><code>""</code> is <code>2</code> characters of code (the two double quotes), but the string contains zero characters.</li>
<li><code>"abc"</code> is <code>5</code> characters of code, but <code>3</code> characters in the string data.</li>
<li><code>"aaa\"aaa"</code> is <code>10</code> characters of code, but the string itself contains six "a" characters and a single, escaped quote character, for a total of <code>7</code> characters in the string data.</li>
<li><code>"\x27"</code> is <code>6</code> characters of code, but the string itself contains just one - an apostrophe (<code>'</code>), escaped using hexadecimal notation.</li>
</ul>
<p>Santa's list is a file that contains many double-quoted string literals, one on each line.  The only escape sequences used are <code>\\</code> (which represents a single backslash), <code>\"</code> (which represents a lone double-quote character), and <code>\x</code> plus two hexadecimal characters (which represents a single character with that ASCII code).</p>
<p>Disregarding the whitespace in the file, what is <em>the number of characters of code for string literals</em> minus <em>the number of characters in memory for the values of the strings</em> in total for the entire file?</p>
<p>For example, given the four strings above, the total number of characters of string code (<code>2 + 5 + 10 + 6 = 23</code>) minus the total number of characters in memory for string values (<code>0 + 3 + 7 + 1 = 11</code>) is <code>23 - 11 = 12</code>.</p>
</article>


In [2]:
import code
import codecs


chars_in_memory_tests = [
    {
        "name": "Example 1",
        "s": r"",
        "expected": 0,
    },
    {
        "name": "Example 2",
        "s": r"abc",
        "expected": 3,
    },
    {
        "name": "Example 3",
        "s": r"aaa\"aaa",
        "expected": 7,
    },
    {
        "name": "Example 4",
        "s": r"\x27",
        "expected": 1,
    },
    {
        "name": "Example 5",
        "s": r"\\",
        "expected": 1,
    },
]


def chars_in_memory(s: str) -> int:
    return len(eval(s))


chars_in_code_tests = [
    {
        "name": "Example 1",
        "s": r"",
        "expected": 2,
    },
    {
        "name": "Example 1",
        "s": r"abc",
        "expected": 5,
    },
    {
        "name": "Example 1",
        "s": r"aaa\"aaa",
        "expected": 10,
    },
    {
        "name": "Example 1",
        "s": r"\x27",
        "expected": 6,
    },
    {
        "name": "Example 5",
        "s": r"\\",
        "expected": 4,
    },
]


def chars_in_code(s: str) -> int:
    return len(s)


run_tests_params(chars_in_memory, chars_in_memory_tests)
run_tests_params(chars_in_code, chars_in_code_tests)

SyntaxError: invalid syntax (<string>, line 0)

In [ ]:
s = r"""
""
"abc"
"aaa\"aaa"
"\x27"
"""


def difference(s: str):
    return sum(chars_in_code(l) - chars_in_memory(l) for l in s.strip().splitlines())


assert difference(s) == 12

# (2 + 5 + 10 + 6 = 23)
# (0 + 3 + 7 + 1 = 11)

In [ ]:
with open("../input/day8.txt", encoding="utf-8") as f:
    s = f.read()

difference(s)

<link href="style.css" rel="stylesheet"></link>
<main>
<article>
<p>Your puzzle answer was <code>1371</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now, let's go the other way.  In addition to finding the number of characters of code, you should now <em>encode each code representation as a new string</em> and find the number of characters of the new encoded representation, including the surrounding double quotes.</p>
<p>For example:</p>
<ul>
<li><code>""</code> encodes to <code>"\"\""</code>, an increase from <code>2</code> characters to <code>6</code>.</li>
<li><code>"abc"</code> encodes to <code>"\"abc\""</code>, an increase from <code>5</code> characters to <code>9</code>.</li>
<li><code>"aaa\"aaa"</code> encodes to <code>"\"aaa\\\"aaa\""</code>, an increase from <code>10</code> characters to <code>16</code>.</li>
<li><code>"\x27"</code> encodes to <code>"\"\\x27\""</code>, an increase from <code>6</code> characters to <code>11</code>.</li>
</ul>
<p>Your task is to find <em>the total number of characters to represent the newly encoded strings</em> minus <em>the number of characters of code in each original string literal</em>. For example, for the strings above, the total encoded length (<code>6 + 9 + 16 + 11 = 42</code>) minus the characters in the original code representation (<code>23</code>, just like in the first part of this puzzle) is <code>42 - 23 = 19</code>.</p>
</article>

</main>


In [ ]:
convert_tests = [
    {
        "name": "Example 1",
        "s": r'""',
        "expected": r"\"\"",
    },
    {
        "name": "Example 2",
        "s": r'"abc"',
        "expected": r"\"abc\"",
    },
    {
        "name": "Example 3",
        "s": r'"aaa\"aaa"',
        "expected": r"\"aaa\\\"aaa\"",
    },
    {
        "name": "Example 4",
        "s": r'"\x27"',
        "expected": r"\"\\x27\"",
    },
    {
        "name": "Example 5",
        "s": r'"\\"',
        "expected": r"\"\\\\\"",
    },
]


def convert(s: str) -> str:
    return s.replace("\\", "\\\\").replace('"', r"\"")


run_tests_params(convert, convert_tests)

In [ ]:
s = r"""
""
"abc"
"aaa\"aaa"
"\x27"
"""


def diffII(s: str) -> int:
    return sum(
        chars_in_code(convert(l)) - chars_in_code(l) + 2 for l in s.strip().splitlines()
    )


diffII(s)

# For example, for the strings above, the total encoded length 
# (6 + 9 + 16 + 11 = 42) 


In [ ]:
with open("../input/day8.txt", encoding="utf-8") as f:
    s = f.read()

diffII(s)

<link href="style.css" rel="stylesheet"></link>

<main>


<p>Your puzzle answer was <code>2117</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2015">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="8/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Matchsticks%22+%2D+Day+8+%2D+Advent+of+Code+2015&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F8&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Matchsticks%22+%2D+Day+8+%2D+Advent+of+Code+2015+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F8';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>